In [ ]:
# import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
# functions to be used
def replace_chars (value):
    """replaces non alpha character with '-''"""
    pattern = r'[^a-zA-Z\.\s]+'
    return re.sub(pattern,'',value).lower()

def last_page_number(li):
    """returns maximum page of the extensiin category"""
    req = requests.get(moz_link+li.select('a')[0].get('href')).text
    return int(BeautifulSoup(req,'xml').select('.Paginate-page-number')[0].getText().split()[-1]) + 1


In [ ]:
# Get Extensions
moz_link = 'https://addons.mozilla.org'
extension_url = moz_link + '/en-US/firefox/extensions/'
extension_url

In [ ]:
req = requests.get(extension_url) # get extension page html file

In [ ]:
master_soup = BeautifulSoup(req.content,'html.parser') # Parse request file as soup

In [ ]:
extension_dict = {} # dictionary for extension links by category

for li in master_soup.select('.Categories-list')[0].select('li'): # selects all categories in extension
    for x in range(1,last_page_number(li)): # gets the number of pages for each category
        link = moz_link+li.select('a')[0].get('href')+f'?page={x}' # links of each page
        category = li.getText() # category names
        try:
            if link in extension_dict[category]: # skip if link is already in the category
                continue
            else:
                extension_dict[category].append(link) 
        except KeyError:
            extension_dict[category] = []
            extension_dict[category].append(link)

In [ ]:
extension_requests = {} # dictionary for soup
for index,(key,links) in enumerate(extension_dict.items()):
    if index < 2: # reduced the value due to memory 
        for link in extension_dict[key]:
            try:
                res = requests.get(link).content
                soup = BeautifulSoup(res,'html.parser')
                extension_requests[key].append(soup)
                
            except KeyError: # when key error occurs,  runs only once for each category
                extension_requests[key] = []
                res = requests.get(link).content
                soup = BeautifulSoup(res,'html.parser')
                extension_requests[key].append(soup)


In [ ]:
category_list = []
names = []
user_count = []
badge = []
rating = []
description = []
author = []
for category in extension_requests:
    for index,soup in enumerate(extension_requests[category]):
        for x in soup.select('.SearchResult-wrapper'):
            category_list.append(category) # gets extension's category name
            user_count.append(x.select('span.SearchResult-users-text')[0].getText()) # gets user count
            names.append(x.select('a.SearchResult-link')[0].get_text()) # gets extension name
            description.append(x.select('.SearchResult-summary')[0].get_text()) # gets extension summary
            author.append(x.select('h3.SearchResult-author')[0].get_text()) # gets extension author
            rating.append(x.select('.SearchResult-rating')[0].get_text()) # gets extension average rating
            if x.select('.PromotedBadge-label--recommended'): # gets extension bagde 
                badge.append(x.select('.PromotedBadge-label--recommended')[0].get_text())
            else:
                badge.append(None) # return None if the badge is not available

In [ ]:
df1 = pd.DataFrame({
    'name':names,'category':category_list,'ratings':rating,
    'description':description,'author':author,'badge':badge,
    'users':user_count,
})

In [ ]:
df1['category'].value_counts()

In [ ]:
# new_df = {}
# for category in extension_dict:
#     for page in extension_dict[category]:
#         del_list = page.select('.SearchResult-wrapper')
#         for a in del_list:
#             def features_columns():
#                 new_df['category'].append(category)
#                 new_df['name'].append(a.select('a.SearchResult-link')[0].get_text())
#                 new_df['users'].append(a.select('span.SearchResult-users-text')[0].get_text())
#                 new_df['author'].append(a.select('h3.SearchResult-author')[0].get_text())
#                 new_df['summary'].append(a.select('.SearchResult-summary')[0].get_text())
#                 new_df['rating'].append(a.select('.SearchResult-rating')[0].get_text())
#                 if a.select('.PromotedBadge-label--recommended'):
#                     new_df['recommended'].append(a.select('.PromotedBadge-label--recommended')[0].get_text())
#                 else: new_df['recommended'].append(None)
#             try:    
#                 features_columns()
#             except KeyError:
#                 new_df['name'] = []
#                 new_df['users'] = []
#                 new_df['author'] = []
#                 new_df['summary'] = []
#                 new_df['rating'] = []
#                 new_df['recommended'] = []                
#                 new_df['category'] = []
#                 features_columns()
    
# print('Done!')

In [ ]:
df1[df1.duplicated()]

In [ ]:
df1[df1['category'] == 'Accessibility'][df1[df1['category'] == 'Accessibility'].duplicated()]

In [ ]:
df1['users'] = df1['users'].apply(lambda x:int(x.split()[0].replace(',','')))

In [ ]:
df1['users'].max()

In [ ]:
gaps

In [ ]:
gaps = '→ Price History charts \n→ Price Drop & '
gaps = '工具栏添加一个Bilibili图标，用于提示并快速查看更新视频、直播等信息'
gaps.encode('ascii',errors='ignore')
# replace_chars(gaps)

In [ ]:
df1.sort_values('users',ascending=False).iloc[217]